In [ ]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
import re

In [ ]:
nfl_plays_raw = spark.sql("SELECT * FROM nfl_plays")

id_cols = ["gameId", "playId"]

target_cols = ["playResult", "passResult", "epa"]

feature_cols = [
    "defendersInTheBox", "numberOfPassRushers", "offenseFormation",
    "personnelO", "personnelD"
]

breakdown_cols = [
    "quarter", "down", "absoluteYardlineNumber", "gameClock",
    "preSnapVisitorScore", "preSnapHomeScore", "possessionTeam"
]

nfl_plays = nfl_plays_raw.selectExpr(id_cols + breakdown_cols + feature_cols +
                                     target_cols)

display(nfl_plays)

gameId,playId,quarter,down,absoluteYardlineNumber,gameClock,preSnapVisitorScore,preSnapHomeScore,possessionTeam,defendersInTheBox,numberOfPassRushers,offenseFormation,personnelO,personnelD,playResult,passResult,epa
2018090600,75,1,1,90,15:00:00,0,0,ATL,7,4,I_FORM,"2 RB, 1 TE, 2 WR","4 DL, 2 LB, 5 DB",10,C,0.261827272178674
2018090600,146,1,1,49,13:10:00,0,0,ATL,7,4,SINGLEBACK,"1 RB, 1 TE, 3 WR","4 DL, 2 LB, 5 DB",0,I,-0.372359818040743
2018090600,168,1,2,49,13:05:00,0,0,ATL,6,4,SHOTGUN,"2 RB, 1 TE, 2 WR","4 DL, 2 LB, 5 DB",0,I,-0.702778658242491
2018090600,190,1,3,49,13:01:00,0,0,ATL,6,5,SHOTGUN,"1 RB, 1 TE, 3 WR","4 DL, 1 LB, 6 DB",33,C,3.04752999500653
2018090600,256,1,3,11,10:59:00,0,0,ATL,8,6,SHOTGUN,"2 RB, 3 TE, 0 WR","6 DL, 3 LB, 2 DB",0,I,-0.842271872651946
2018090600,320,1,2,14,10:10:00,0,0,PHI,7,4,SHOTGUN,"1 RB, 1 TE, 3 WR","3 DL, 3 LB, 5 DB",4,C,-0.344096467462392
2018090600,344,1,3,18,09:24:00,0,0,PHI,6,4,EMPTY,"1 RB, 2 TE, 2 WR","3 DL, 3 LB, 5 DB",0,I,-1.19220828463531
2018090600,402,1,1,54,09:08:00,0,0,ATL,7,4,SINGLEBACK,"1 RB, 1 TE, 3 WR","4 DL, 2 LB, 5 DB",0,I,-0.429862558722865
2018090600,492,1,2,46,07:01:00,0,0,ATL,8,4,SINGLEBACK,"1 RB, 2 TE, 2 WR","4 DL, 2 LB, 5 DB",26,C,1.87980397791624
2018090600,521,1,1,20,06:19:00,0,0,ATL,8,3,SINGLEBACK,"1 RB, 2 TE, 2 WR","4 DL, 2 LB, 5 DB",3,C,0.0456650977161344


## Data cleanup

*   Remove "null" from offenseFormation
*   Rename some ambiguous column names
*   Split the personnelO and personnelD column into invidivual rows

#### Cleanup `offenseFormation` column

In [ ]:
# Confirm possible values for offenseFormation
display(
    nfl_plays.groupBy("offenseFormation").count().selectExpr(
        "offenseFormation"))

offenseFormation
PISTOL
null
EMPTY
WILDCAT
SINGLEBACK
I_FORM
JUMBO
SHOTGUN


In [ ]:
display(nfl_plays.groupBy("passResult").count())

passResult,count
C,11370
IN,420
S,1308
I,6135
null,2
R,4


In [ ]:
# Replace the null value in "offenseFormation" with "EMPTY", and set the passResult of null to "I"
def replace_null_in_offense_formation(offenseFormation):
    if (offenseFormation == "null"):
        return "EMPTY"
    if (not offenseFormation):
        return "EMPTY"
    return offenseFormation


replace_null_offForm_udf = udf(
    lambda offenseFormation: replace_null_in_offense_formation(offenseFormation
                                                              ), StringType())

nfl_plays = nfl_plays.withColumn(
    "offenseFormation", replace_null_offForm_udf(nfl_plays.offenseFormation))
nfl_plays = nfl_plays.fillna("I", "passResult")
display(nfl_plays)

gameId,playId,quarter,down,absoluteYardlineNumber,gameClock,preSnapVisitorScore,preSnapHomeScore,possessionTeam,defendersInTheBox,numberOfPassRushers,offenseFormation,personnelO,personnelD,playResult,passResult,epa
2018090600,75,1,1,90,15:00:00,0,0,ATL,7,4,I_FORM,"2 RB, 1 TE, 2 WR","4 DL, 2 LB, 5 DB",10,C,0.261827272178674
2018090600,146,1,1,49,13:10:00,0,0,ATL,7,4,SINGLEBACK,"1 RB, 1 TE, 3 WR","4 DL, 2 LB, 5 DB",0,I,-0.372359818040743
2018090600,168,1,2,49,13:05:00,0,0,ATL,6,4,SHOTGUN,"2 RB, 1 TE, 2 WR","4 DL, 2 LB, 5 DB",0,I,-0.702778658242491
2018090600,190,1,3,49,13:01:00,0,0,ATL,6,5,SHOTGUN,"1 RB, 1 TE, 3 WR","4 DL, 1 LB, 6 DB",33,C,3.04752999500653
2018090600,256,1,3,11,10:59:00,0,0,ATL,8,6,SHOTGUN,"2 RB, 3 TE, 0 WR","6 DL, 3 LB, 2 DB",0,I,-0.842271872651946
2018090600,320,1,2,14,10:10:00,0,0,PHI,7,4,SHOTGUN,"1 RB, 1 TE, 3 WR","3 DL, 3 LB, 5 DB",4,C,-0.344096467462392
2018090600,344,1,3,18,09:24:00,0,0,PHI,6,4,EMPTY,"1 RB, 2 TE, 2 WR","3 DL, 3 LB, 5 DB",0,I,-1.19220828463531
2018090600,402,1,1,54,09:08:00,0,0,ATL,7,4,SINGLEBACK,"1 RB, 1 TE, 3 WR","4 DL, 2 LB, 5 DB",0,I,-0.429862558722865
2018090600,492,1,2,46,07:01:00,0,0,ATL,8,4,SINGLEBACK,"1 RB, 2 TE, 2 WR","4 DL, 2 LB, 5 DB",26,C,1.87980397791624
2018090600,521,1,1,20,06:19:00,0,0,ATL,8,3,SINGLEBACK,"1 RB, 2 TE, 2 WR","4 DL, 2 LB, 5 DB",3,C,0.0456650977161344


#### Rename `playResult` and `epa`

In [ ]:
nfl_plays = nfl_plays.withColumnRenamed("playResult",
                                        "netYardsGained").withColumnRenamed(
                                            "epa", "expectedPointsAdded")

display(nfl_plays)

gameId,playId,quarter,down,absoluteYardlineNumber,gameClock,preSnapVisitorScore,preSnapHomeScore,possessionTeam,defendersInTheBox,numberOfPassRushers,offenseFormation,personnelO,personnelD,netYardsGained,passResult,expectedPointsAdded
2018090600,75,1,1,90,15:00:00,0,0,ATL,7,4,I_FORM,"2 RB, 1 TE, 2 WR","4 DL, 2 LB, 5 DB",10,C,0.261827272178674
2018090600,146,1,1,49,13:10:00,0,0,ATL,7,4,SINGLEBACK,"1 RB, 1 TE, 3 WR","4 DL, 2 LB, 5 DB",0,I,-0.372359818040743
2018090600,168,1,2,49,13:05:00,0,0,ATL,6,4,SHOTGUN,"2 RB, 1 TE, 2 WR","4 DL, 2 LB, 5 DB",0,I,-0.702778658242491
2018090600,190,1,3,49,13:01:00,0,0,ATL,6,5,SHOTGUN,"1 RB, 1 TE, 3 WR","4 DL, 1 LB, 6 DB",33,C,3.04752999500653
2018090600,256,1,3,11,10:59:00,0,0,ATL,8,6,SHOTGUN,"2 RB, 3 TE, 0 WR","6 DL, 3 LB, 2 DB",0,I,-0.842271872651946
2018090600,320,1,2,14,10:10:00,0,0,PHI,7,4,SHOTGUN,"1 RB, 1 TE, 3 WR","3 DL, 3 LB, 5 DB",4,C,-0.344096467462392
2018090600,344,1,3,18,09:24:00,0,0,PHI,6,4,EMPTY,"1 RB, 2 TE, 2 WR","3 DL, 3 LB, 5 DB",0,I,-1.19220828463531
2018090600,402,1,1,54,09:08:00,0,0,ATL,7,4,SINGLEBACK,"1 RB, 1 TE, 3 WR","4 DL, 2 LB, 5 DB",0,I,-0.429862558722865
2018090600,492,1,2,46,07:01:00,0,0,ATL,8,4,SINGLEBACK,"1 RB, 2 TE, 2 WR","4 DL, 2 LB, 5 DB",26,C,1.87980397791624
2018090600,521,1,1,20,06:19:00,0,0,ATL,8,3,SINGLEBACK,"1 RB, 2 TE, 2 WR","4 DL, 2 LB, 5 DB",3,C,0.0456650977161344


#### Convert `passResult` and `offenseFormation` to numerical columns

In [ ]:
# nfl_plays = nfl_plays.drop(col("passResultIndex"))
# nfl_plays = nfl_plays.drop(col("offenseFormationIndex"))

In [ ]:
nfl_plays = StringIndexer(
    inputCol="passResult",
    outputCol="passResultIndex").fit(nfl_plays).transform(nfl_plays)
nfl_plays = StringIndexer(
    inputCol="offenseFormation",
    outputCol="offenseFormationIndex").fit(nfl_plays).transform(nfl_plays)

display(nfl_plays)

gameId,playId,quarter,down,absoluteYardlineNumber,gameClock,preSnapVisitorScore,preSnapHomeScore,possessionTeam,defendersInTheBox,numberOfPassRushers,offenseFormation,personnelO,personnelD,netYardsGained,passResult,expectedPointsAdded,passResultIndex,offenseFormationIndex
2018090600,75,1,1,90,15:00:00,0,0,ATL,7,4,I_FORM,"2 RB, 1 TE, 2 WR","4 DL, 2 LB, 5 DB",10,C,0.261827272178674,0.0,3.0
2018090600,146,1,1,49,13:10:00,0,0,ATL,7,4,SINGLEBACK,"1 RB, 1 TE, 3 WR","4 DL, 2 LB, 5 DB",0,I,-0.372359818040743,1.0,1.0
2018090600,168,1,2,49,13:05:00,0,0,ATL,6,4,SHOTGUN,"2 RB, 1 TE, 2 WR","4 DL, 2 LB, 5 DB",0,I,-0.702778658242491,1.0,0.0
2018090600,190,1,3,49,13:01:00,0,0,ATL,6,5,SHOTGUN,"1 RB, 1 TE, 3 WR","4 DL, 1 LB, 6 DB",33,C,3.04752999500653,0.0,0.0
2018090600,256,1,3,11,10:59:00,0,0,ATL,8,6,SHOTGUN,"2 RB, 3 TE, 0 WR","6 DL, 3 LB, 2 DB",0,I,-0.842271872651946,1.0,0.0
2018090600,320,1,2,14,10:10:00,0,0,PHI,7,4,SHOTGUN,"1 RB, 1 TE, 3 WR","3 DL, 3 LB, 5 DB",4,C,-0.344096467462392,0.0,0.0
2018090600,344,1,3,18,09:24:00,0,0,PHI,6,4,EMPTY,"1 RB, 2 TE, 2 WR","3 DL, 3 LB, 5 DB",0,I,-1.19220828463531,1.0,2.0
2018090600,402,1,1,54,09:08:00,0,0,ATL,7,4,SINGLEBACK,"1 RB, 1 TE, 3 WR","4 DL, 2 LB, 5 DB",0,I,-0.429862558722865,1.0,1.0
2018090600,492,1,2,46,07:01:00,0,0,ATL,8,4,SINGLEBACK,"1 RB, 2 TE, 2 WR","4 DL, 2 LB, 5 DB",26,C,1.87980397791624,0.0,1.0
2018090600,521,1,1,20,06:19:00,0,0,ATL,8,3,SINGLEBACK,"1 RB, 2 TE, 2 WR","4 DL, 2 LB, 5 DB",3,C,0.0456650977161344,0.0,1.0


#### Transform personnelO and personnelD to be their on columns

Use the most common positions for columns.

In [ ]:
# Replace the null value in "offenseFormation" with "EMPTY"
def map_number_position(position_code, personnel):
    if not personnel:
        return int(0)
    personnel_match = re.findall(r'\d+' + " " + position_code, personnel)
    if (len(personnel_match) == 0 or len(personnel_match) > 1):
        return int(0)

    match = re.findall(r'\d+', personnel_match[0])[0]
    return int(match)


map_rb_personnel_udf = udf(
    lambda personnel: map_number_position("RB", personnel), IntegerType())
map_te_personnel_udf = udf(
    lambda personnel: map_number_position("TE", personnel), IntegerType())
map_wr_personnel_udf = udf(
    lambda personnel: map_number_position("WR", personnel), IntegerType())

map_dl_personnel_udf = udf(
    lambda personnel: map_number_position("DL", personnel), IntegerType())
map_lb_personnel_udf = udf(
    lambda personnel: map_number_position("LB", personnel), IntegerType())
map_db_personnel_udf = udf(
    lambda personnel: map_number_position("DB", personnel), IntegerType())

nfl_plays = nfl_plays.withColumn("num_offense_runningbacks",
                                 map_rb_personnel_udf(nfl_plays.personnelO))
nfl_plays = nfl_plays.withColumn("num_offense_tightends",
                                 map_te_personnel_udf(nfl_plays.personnelO))
nfl_plays = nfl_plays.withColumn("num_offense_widereceivers",
                                 map_wr_personnel_udf(nfl_plays.personnelO))

nfl_plays = nfl_plays.withColumn("num_def_defline",
                                 map_dl_personnel_udf(nfl_plays.personnelD))
nfl_plays = nfl_plays.withColumn("num_def_linebackers",
                                 map_lb_personnel_udf(nfl_plays.personnelD))
nfl_plays = nfl_plays.withColumn("num_def_defbacks",
                                 map_db_personnel_udf(nfl_plays.personnelD))

display(nfl_plays)

gameId,playId,quarter,down,absoluteYardlineNumber,gameClock,preSnapVisitorScore,preSnapHomeScore,possessionTeam,defendersInTheBox,numberOfPassRushers,offenseFormation,personnelO,personnelD,netYardsGained,passResult,expectedPointsAdded,passResultIndex,offenseFormationIndex,num_offense_runningbacks,num_offense_tightends,num_offense_widereceivers,num_def_defline,num_def_linebackers,num_def_defbacks
2018090600,75,1,1,90,15:00:00,0,0,ATL,7,4,I_FORM,"2 RB, 1 TE, 2 WR","4 DL, 2 LB, 5 DB",10,C,0.261827272178674,0.0,3.0,2,1,2,4,2,5
2018090600,146,1,1,49,13:10:00,0,0,ATL,7,4,SINGLEBACK,"1 RB, 1 TE, 3 WR","4 DL, 2 LB, 5 DB",0,I,-0.372359818040743,1.0,1.0,1,1,3,4,2,5
2018090600,168,1,2,49,13:05:00,0,0,ATL,6,4,SHOTGUN,"2 RB, 1 TE, 2 WR","4 DL, 2 LB, 5 DB",0,I,-0.702778658242491,1.0,0.0,2,1,2,4,2,5
2018090600,190,1,3,49,13:01:00,0,0,ATL,6,5,SHOTGUN,"1 RB, 1 TE, 3 WR","4 DL, 1 LB, 6 DB",33,C,3.04752999500653,0.0,0.0,1,1,3,4,1,6
2018090600,256,1,3,11,10:59:00,0,0,ATL,8,6,SHOTGUN,"2 RB, 3 TE, 0 WR","6 DL, 3 LB, 2 DB",0,I,-0.842271872651946,1.0,0.0,2,3,0,6,3,2
2018090600,320,1,2,14,10:10:00,0,0,PHI,7,4,SHOTGUN,"1 RB, 1 TE, 3 WR","3 DL, 3 LB, 5 DB",4,C,-0.344096467462392,0.0,0.0,1,1,3,3,3,5
2018090600,344,1,3,18,09:24:00,0,0,PHI,6,4,EMPTY,"1 RB, 2 TE, 2 WR","3 DL, 3 LB, 5 DB",0,I,-1.19220828463531,1.0,2.0,1,2,2,3,3,5
2018090600,402,1,1,54,09:08:00,0,0,ATL,7,4,SINGLEBACK,"1 RB, 1 TE, 3 WR","4 DL, 2 LB, 5 DB",0,I,-0.429862558722865,1.0,1.0,1,1,3,4,2,5
2018090600,492,1,2,46,07:01:00,0,0,ATL,8,4,SINGLEBACK,"1 RB, 2 TE, 2 WR","4 DL, 2 LB, 5 DB",26,C,1.87980397791624,0.0,1.0,1,2,2,4,2,5
2018090600,521,1,1,20,06:19:00,0,0,ATL,8,3,SINGLEBACK,"1 RB, 2 TE, 2 WR","4 DL, 2 LB, 5 DB",3,C,0.0456650977161344,0.0,1.0,1,2,2,4,2,5


Create a joint variable comparing number of Defensive Backs and the number of Receivers (Tight end, wide receiver)

In [ ]:
from pyspark.sql.functions import col

nfl_plays = nfl_plays.withColumn(
    "delta_defline_offreceivers",
    col("num_def_defbacks") -
    (col("num_offense_tightends") + col("num_offense_widereceivers")))
display(nfl_plays)

gameId,playId,quarter,down,absoluteYardlineNumber,gameClock,preSnapVisitorScore,preSnapHomeScore,possessionTeam,defendersInTheBox,numberOfPassRushers,offenseFormation,personnelO,personnelD,netYardsGained,passResult,expectedPointsAdded,passResultIndex,offenseFormationIndex,num_offense_runningbacks,num_offense_tightends,num_offense_widereceivers,num_def_defline,num_def_linebackers,num_def_defbacks,delta_defline_offreceivers
2018090600,75,1,1,90,15:00:00,0,0,ATL,7,4,I_FORM,"2 RB, 1 TE, 2 WR","4 DL, 2 LB, 5 DB",10,C,0.261827272178674,0.0,3.0,2,1,2,4,2,5,2
2018090600,146,1,1,49,13:10:00,0,0,ATL,7,4,SINGLEBACK,"1 RB, 1 TE, 3 WR","4 DL, 2 LB, 5 DB",0,I,-0.372359818040743,1.0,1.0,1,1,3,4,2,5,1
2018090600,168,1,2,49,13:05:00,0,0,ATL,6,4,SHOTGUN,"2 RB, 1 TE, 2 WR","4 DL, 2 LB, 5 DB",0,I,-0.702778658242491,1.0,0.0,2,1,2,4,2,5,2
2018090600,190,1,3,49,13:01:00,0,0,ATL,6,5,SHOTGUN,"1 RB, 1 TE, 3 WR","4 DL, 1 LB, 6 DB",33,C,3.04752999500653,0.0,0.0,1,1,3,4,1,6,2
2018090600,256,1,3,11,10:59:00,0,0,ATL,8,6,SHOTGUN,"2 RB, 3 TE, 0 WR","6 DL, 3 LB, 2 DB",0,I,-0.842271872651946,1.0,0.0,2,3,0,6,3,2,-1
2018090600,320,1,2,14,10:10:00,0,0,PHI,7,4,SHOTGUN,"1 RB, 1 TE, 3 WR","3 DL, 3 LB, 5 DB",4,C,-0.344096467462392,0.0,0.0,1,1,3,3,3,5,1
2018090600,344,1,3,18,09:24:00,0,0,PHI,6,4,EMPTY,"1 RB, 2 TE, 2 WR","3 DL, 3 LB, 5 DB",0,I,-1.19220828463531,1.0,2.0,1,2,2,3,3,5,1
2018090600,402,1,1,54,09:08:00,0,0,ATL,7,4,SINGLEBACK,"1 RB, 1 TE, 3 WR","4 DL, 2 LB, 5 DB",0,I,-0.429862558722865,1.0,1.0,1,1,3,4,2,5,1
2018090600,492,1,2,46,07:01:00,0,0,ATL,8,4,SINGLEBACK,"1 RB, 2 TE, 2 WR","4 DL, 2 LB, 5 DB",26,C,1.87980397791624,0.0,1.0,1,2,2,4,2,5,1
2018090600,521,1,1,20,06:19:00,0,0,ATL,8,3,SINGLEBACK,"1 RB, 2 TE, 2 WR","4 DL, 2 LB, 5 DB",3,C,0.0456650977161344,0.0,1.0,1,2,2,4,2,5,1


In [ ]:
spark.conf.set(
    "spark.sql.legacy.allowCreatingManagedTableUsingNonemptyLocation", "true")
nfl_plays.write.saveAsTable("nfl_plays_cleaned", mode="overwrite")